In [2]:
class Datasource():
    def __init__(self):
        self.is_initialized = False
        self.is_deactivated = False
    
    def set_source(self, mode, source):
        self.mode = mode
        self.source = source
        self.is_initialized = True
        
        if self.mode == 'db':
            import sqlite3
            self.conn = sqlite3.connect(self.source[0])
            self.tablename = self.source[1]
            self.source = self.conn.cursor()
            print(self.source)
            self.counter = 0

    def angles(self):
        if self.mode == 'wired':  # Currently only works with one sensor; use a multiplexer for multiple sensor units (see MotioSuit)
            angles = self.source.readline()[:-3].decode('UTF-8').split(',')
            return angles

        elif self.mode == 'prerecorded':  # replaced by BVH export utility
            pass

        else:
            self.source.execute('SELECT * from ' + self.tablename + ' WHERE generation=' + str(self.counter))
            angles = self.source.fetchall()
            if len(angles) == 0:
                self.is_deactivated = True
                self.max_gen = self.counter
            else:
                self.counter += 1
                return angles

    def last_angle(self):
        if self.mode == 'prerecorded' and self.counter >= 0:
            pass
        elif self.mode == 'db' and self.counter >= 0:
            self.counter -= 1
            self.source.execute('SELECT * from ' + self.tablename + ' WHERE generation=' + str(self.counter))
            angles = self.source.fetchall()
            return angles

    def next_angle(self):
        if self.mode == 'prerecorded' and self.counter <= self.max_gen:
            pass
        elif self.mode == 'db' and self.counter <= self.max_gen:
            self.counter += 1
            self.source.execute('SELECT * from ' + self.tablename + ' WHERE generation=' + str(self.counter))
            angles = self.source.fetchall()
            return angles

This example illustrates how sensor readings can be used to directly animate a blender model. Since every sensor configuration is different, the scripts probably have to be adjusted for these configurations. 

In [1]:
import sys
import argparse
import bge  # NOTE: A headless version of blender was compiled from source; if unavailable, use this inside blender
import bpy # Unified interface for all data sources
import math
import mathutils
import os.path
import serial
import time

# TODO: Add more options as the project moves on

source = Datasource()

if '--' in sys.argv:

    argv = sys.argv[sys.argv.index('--') + 1:]  # get all arguments after '--'
    args = parser.parse_args(argv)
    
    print(argv)
    
    if args['mode'] == 'wired':
        sp = args['serialport']
        from serial import serialwin32
        ser = serialwin32.Serial(port, 115200)
        source.set_source('serial', ser)

    elif args['mode'] == 'wireless':
        port = args['port']
        from server import Server
        server = Server(port)  # TODO: Adapt new server class for this
        source.set_source('server', server)

    elif args['mode'] == 'prerecorded':
        path = args['path']
        source.set_source('file', path)

    elif args['mode'] == 'db':
        db = args['db']
        source.set_source('db', (db, tablename))

else:
    mode = input('Please select a data source: [1] for wired transmissions, [2] for prerecorded transmissions and [3] for databases ')
    
    if mode == '1':
        from serial import serialwin32
        port = input('Please enter the serial port to receive data from: ')
        ser = serialwin32.Serial(port, 115200)
        source.set_source('serial', ser)

    elif mode == '2':
        path = input('Please enter the file to read data form: ')
        source.set_source('file', path)       

    elif mode == '3':
        db = input('Please enter the db to read data from: ')
        tablename = input('Please enter the table to read from: ')
        source.set_source('db', (db, tablename))

scene = bge.logic.getCurrentScene()
main_arm = scene.objects.get('Armature')
ob = bge.logic.getCurrentController().owner


def updateAngles():
    if source.is_initialized and source.is_deactivated is not True:
        
        angles = source.angles()

        # Debug
        print(angles)
        angle = mathutils.Vector((angles[0][1], angles[0][2], angles[0][3], angles[0][4]))
        #correction = mathutils.Quaternion((1.0, 0.0, 0.0), math.radians(90))
        angle_corrected = angle 
        ob.channels['upperLegR'].rotation_quaternion = angle_corrected

        # TODO: Create some kind of sensor to joint mapping
        ob.update()
        time.sleep(0.001)

def request_last_angle():
    if source.is_deactivated:
        angles = source.last_angle()
        # Debug
        print(angles)
        angle = mathutils.Vector((angles[0][1], angles[0][2], angles[0][3], angles[0][4]))
        #correction = mathutils.Quaternion((1.0, 0.0, 0.0), math.radians(90))
        
        # NOTE: Depending on the configuration used, this step requires manual sensor-joint mapping;
        # An example is given below:
        angle_corrected = angle 
        ob.channels['upperLegR'].rotation_quaternion = angle_corrected

        
        ob.update()      

def request_next_angle():
    if source.is_deactivated:
        angles = source.next_angle()        

        # Debug
        print(angles)
        angle = mathutils.Vector((angles[0][1], angles[0][2], angles[0][3], angles[0][4]))
        #correction = mathutils.Quaternion((1.0, 0.0, 0.0), math.radians(90))
        angle_corrected = angle 
        ob.channels['upperLegR'].rotation_quaternion = angle_corrected

        # TODO: Create some kind of sensor to joint mapping
        ob.update()    

ModuleNotFoundError: No module named 'bge'